In [ ]:
import pandas as pd
from pyspark import SparkContext
from operator import add
import sys
sc = SparkContext('local','pyspark')
class Firewall:
    
    def __init__(self, path):
        
#         self.content = pd.read_csv(str(path), sep = ',', engine = 'python')

        def splt0(x):
            temp = x[0].find('-')
            if temp == -1:
                return int(x[0]), int(x[0])
            else:
                return int(x[0][:temp]), int(x[0][temp+1, :])
            
        def splt1(x):
            temp = x[1].find('-')      
            b=list(x[1])
            l = []
            if temp == -1:         
                for item in enumerate(b):
                    if item[1]=='.':
                        l.append(item[0])
                for i in range(3-l[0]):
                    b.insert(0,'0')
                for i in range(4-l[1]+l[0]):
                    b.insert(4, '0')
                for i in range(4-l[2]+l[1]):
                    b.insert(8, '0')
                for i in range(15-len(b)):
                    b.insert(12, '0')
                address = "".join(b)
                return address, address
            else:
                bs = [b[:temp], b[temp+1:]]
                for j in range(2):
                    for item in enumerate(bs[j]):
                        if item[1]=='.':
                            l.append(item[0])
                    for i in range(3-l[0]):
                        bs[j].insert(0,'0')
                    for i in range(4-l[1]+l[0]):
                        bs[j].insert(4, '0')
                    for i in range(4-l[2]+l[1]):
                        bs[j].insert(8, '0')
                    for i in range(15-len(b)):
                        bs[j].insert(12, '0')
                    address0 = "".join(bs[0])
                    address1 = "".join(bs[1])
                return address0, address1
            
        self.out = sc.textFile(path, 3).map(lambda x: x.split(','))
        self.int = self.out.filter(lambda x: x[0]=='inbound').map(lambda x: list(x[1:]))
        self.out = self.out.filter(lambda x: x[0]=='outbound').map(lambda x: list(x[1:]))
        
        self.inu = self.int.filter(lambda x: x[1]=='udp').map(lambda x: list(x[1:])).map(splt0).map(splt1).collect()
        self.int = self.int.filter(lambda x: x[1]=='tcp').map(lambda x: list(x[1:])).map(splt0).map(splt1).collect()
        self.ouu = self.out.filter(lambda x: x[1]=='udp').map(lambda x: list(x[1:])).map(splt0).map(splt1).collect()
        self.out = self.out.filter(lambda x: x[1]=='tcp').map(lambda x: list(x[1:])).map(splt0).map(splt1).collect()        
            
    def accept_packet(self, direction, protocol, port, ip):
        if direction=='inbound':
            if protocol=='tcp':
                for i in self.int:
                    if i[0]<=port and port<=i[1] and i[2]<=ip and ip<=i[3]:
                        return True
                return False
            else:
                for i in self.inu:
                    if i[0]<=port and port<=i[1] and i[2]<=ip and ip<=i[3]:
                        return True
                return False
        else:
             if protocol=='tcp':
                for i in self.out:
                    if i[0]<=port and port<=i[1] and i[2]<=ip and ip<=i[3]:
                        return True
                return False
            else:
                for i in self.ouu:
                    if i[0]<=port and port<=i[1] and i[2]<=ip and ip<=i[3]:
                        return True
                return False